**<h1>Setup and data processing<h1>**

---



<h3>Package installation<h3>


In [ ]:
%pip install pandas
%pip install python-binance
%pip install ta
%pip install matplotlib
%pip install numpy

<h3>Imports<h3>

In [ ]:
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

<h3>Specs<h3>

In [ ]:
name_base = "ETH"
name_quote = "USDT"
timeframe = "1d"

starting_date = "01 january 2017"
ending_date = None

n_bars = 10

<h3>Data<h3>

In [ ]:
# download the coin info
symbol = name_base+name_quote
info = Client().get_historical_klines(symbol, timeframe, starting_date)
# info = Client(tld='us').get_historical_klines(symbol, timeframe, starting_date) # if in US or Japan.

# storing it into a pandas data frame
data_dl = pd.DataFrame(info, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# keeping only the necessary columns
data_dl.drop(columns=data_dl.columns.difference(['timestamp', 'open', 'high', 'low', 'close']), inplace=True)

# formating the numbers
data_dl['timestamp'] = pd.to_datetime(data_dl['timestamp'], unit='ms')
data_dl['open'] = pd.to_numeric(data_dl['open'])
data_dl['high'] = pd.to_numeric(data_dl['high'])
data_dl['low'] = pd.to_numeric(data_dl['low'])
data_dl['close'] = pd.to_numeric(data_dl['close'])
data_dl

**<h1>Optimisation<h1>**


---

<h3>Entry condition<h3>

In [ ]:
def entry(row):
    return row['macd_hist'] > 0

<h3>Loop<h3>

In [ ]:
initial_wallet = 1000

array_s = np.linspace(2, 50, int((50-2)/2+1), dtype=int)
array_f = np.linspace(2, 50, int((50-2)/2+1), dtype=int)

results = []
for s in array_s:
    for f in array_f:
        if f > s: 
            
            ### indicators
            data = data_dl.copy()
            MACD = ta.trend.MACD(data['close'], window_slow = s, window_fast = f, window_sign = 9)
            data['macd_hist'] = MACD.macd_diff()
            data.dropna(inplace=True)

            ### initialisations
            quote = initial_wallet
            base = 0
            orders = []
            data['wallet'] = ''
            ct = 0
            buy_ready = True

            ### backtest loop
            for index, row in data.iterrows():

                value = row['close']

                if entry(row) and buy_ready:
                    ct = index
                    base = quote / value
                    quote = 0
                    buy_ready = False
                    orders.append({'side': 'entry', 'wallet': base * value})

                # Exit after n_bars
                if index == ct + n_bars:
                    quote = base * value
                    base = 0
                    buy_ready = True
                    ct = 0
                    orders.append({'side': 'exit', 'wallet': quote})

           
            ### analysing the trades
            orders = pd.DataFrame(orders, columns=['side', 'wallet'])
            orders['PnL%'] = orders['wallet'].pct_change()*100
            orders.at[0, 'PnL%'] = (orders.iloc[0]['wallet']-initial_wallet)/initial_wallet*100
            orders.loc[orders['side']=='entry','PnL%'] = None
            orders['Win'] = ''
            orders.loc[orders['PnL%']>0,'Win'] = 'Yes'
            orders.loc[orders['PnL%']<=0,'Win'] = 'No'
            if 'No' in orders['Win'].values and 'Yes' in orders['Win'].values:
                n_pos = orders['Win'].value_counts()['Yes']
                n_neg = orders['Win'].value_counts()['No']
                winrate = n_pos / (n_neg + n_pos) * 100
            elif 'No' not in orders['Win'].values and 'Yes' not in orders['Win'].values:
                winrate = 0
            else:
                winrate = 100

            res = {
                  'fastEMA': f,
                  'slowEMA': s,
                  'winrate': winrate,
                  'avg_trades': round(orders['PnL%'].mean(),2),
            }
            results.append(res)

            print(f"slow EMA = {res['slowEMA']}; fast EMA = {res['fastEMA']}")
 
 

**<h1>Results<h1>**


---



In [ ]:
# results into a dataframe
results = pd.DataFrame(results, columns=['fastEMA', 'slowEMA', 'wallet', 'winrate', 'avg_trades'])

# color map for plots
cmap_reversed = cm.get_cmap('PuOr_r')

# specs
obj_x = 'slowEMA'
obj_y = 'fastEMA' 
label_x = 'slow EMA'
label_y = 'fast EMA'

<h3>Winrate<h3>

In [ ]:
plt.figure(figsize=(8, 6))
sc = plt.scatter(results[obj_x], results[obj_y], c=results["winrate"], cmap=cmap_reversed, s=25)
plt.xlabel(label_x, fontsize=24)
plt.ylabel(label_y, fontsize=24)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
cb = plt.colorbar(sc)
cb.set_label(label = 'Winrate (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.tight_layout()

<h3>Average profit per trade<h3>

In [ ]:
plt.figure(figsize=(8, 6))
sc = plt.scatter(results[obj_x], results[obj_y], c=results["avg_trades"], cmap=cmap_reversed, s=25)
plt.xlabel(label_x, fontsize=24)
plt.ylabel(label_y, fontsize=24)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
cb = plt.colorbar(sc)
cb.set_label(label = 'Average Trade Profits (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.tight_layout()